# 4 Panda Library

## 4.1 Reading csv and basic info

In [2]:
import pandas as pd

"""
Get file from map data
Datatype is "dataframe"
csv files are most common (comma seperated values)
"""

covid_df = pd.read_csv('data/italy-covid-daywise.csv') 

In [20]:
covid_df.info()                        # basic info about rows, columns, datatypes
covid_df.describe()                    # stat info about the data (count, average... per column)
list_of_columns = covid_df.columns     # get list of column names
shape = covid_df.shape                 # numbers of rows and columns as a tuple

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        248 non-null    object 
 1   new_cases   248 non-null    float64
 2   new_deaths  248 non-null    float64
 3   new_tests   135 non-null    float64
dtypes: float64(3), object(1)
memory usage: 7.9+ KB


,new_cases,new_deaths,new_tests
count,248.000000,248.000000,135.000000
mean,1094.818548,143.133065,31699.674074
std,1554.508002,227.105538,11622.209757
min,-148.000000,-31.000000,7841.000000
25%,123.000000,3.000000,25259.000000
50%,342.000000,17.000000,29545.000000
75%,1371.750000,175.250000,37711.000000
max,6557.000000,971.000000,95273.000000


## 4.2 Retrieving data from a dataframe

In [ ]:
# Pandas dataframe format is simliar to the dictionary
covid_data_dict = {
    'date':       ['2020-08-30', '2020-08-31', '2020-09-01', '2020-09-02', '2020-09-03'],
    'new_cases':  [1444, 1365, 996, 975, 1326],
    'new_deaths': [1, 4, 6, 8, 6],
    'new_tests': [53541, 42583, 54395, None, None]
}

covid_df.new_cases                           # get column new cases
cases_df = covid_df[['date', 'new_cases']]   # make a dataframe with only date and new cases as columns
covid_df.at[246, 'new_cases']                # get value of new cases column at place 246

covid_df.loc[243]                            # get row 243 from data
covid_df.loc[108:113]                        # get rows between row 108 and 113
covid_df.head(5)                             # get first 5 rows
covid_df.tail(4)                             # get latest 4 rows
covid_df.sample(10)                          # get random sample of 10 rows
flowers_df.species.unique()                  # different unique options in column

covid_df.new_tests.first_valid_index()       # get first index without a Nan value

covid_df_copy = covid_df.copy()              # copy to seperate (new) dataframe

## 4.3 Editing data from a dataframe

In [ ]:
"""
Als er een vreemd getal is bv -148 kijk na de data er omheen, documentatie 
(is het mss een correctie voor mistelling etc) en pas dan pas aan:
average of last and following, replace with Nan, delete row...
"""

high_new_cases = covid_df.new_cases > 1000                            # return new boolean column, true if new cases > 1000
covid_df[high_new_cases]                                              # add boolean row in dataframe
high_ratio = covid_df[covid_df.new_cases / covid_df.new_tests > 0.5]  # element wise editing, true if > 0,5

covid_df['positive_rate'] = covid_df.new_cases / covid_df.new_tests   # add new key/column that has ratio of positive tests
covid_df.drop(columns=['positive_rate'], inplace=True)                # delete a column

covid_df.sort_values('new_cases', ascending=False).head(10)           # sort new cases en show first 10

covid_df.at[172, 'new_cases'] = covid_df.at[171, 'new_cases']         # replacing a value in a dataframe

In [ ]:
covid_df['date'] = pd.to_datetime(covid_df.date)                                 # change format to date (for splitting)

# make columns of the parts
covid_df['year'] = pd.DatetimeIndex(covid_df.date).year
covid_df['month'] = pd.DatetimeIndex(covid_df.date).month
covid_df['day'] = pd.DatetimeIndex(covid_df.date).day
covid_df['weekday'] = pd.DatetimeIndex(covid_df.date).weekday

covid_df_may = covid_df[covid_df.month == 5]                                     # Query the rows for May
covid_df_may_metrics = covid_df_may[['new_cases', 'new_deaths', 'new_tests']]    # Extract subset of columns to be aggregated

covid_may_totals = covid_df_may_metrics.sum()                                    # Get the column-wise sum
covid_df[covid_df.month == 5][['new_cases', 'new_deaths', 'new_tests']].sum()

covid_df.new_cases.mean()                                                        # Overall average
covid_df[covid_df.weekday == 6].new_cases.mean()                                 # Average for Sundays

## 4.4 Grouping and combining data

In [ ]:
covid_df.groupby('month')[['new_cases', 'new_deaths']].sum()                     # group every month and sum them
covid_df.groupby('weekdays')[['new_cases', 'new_deaths', 'new_tests']].mean()    # group every weekday and take the average

covid_df['total_cases'] = covid_df.new_cases.cumsum()                            # cumulative sum of cases

In [17]:
locations_df = pd.read_csv('data/locations.csv')
locations_df

,location,continent,population,life_expectancy,hospital_beds_per_thousand,gdp_per_capita
0,Afghanistan,Asia,3.892834e+07,64.83,0.500,1803.987
1,Albania,Europe,2.877800e+06,78.57,2.890,11803.431
2,Algeria,Africa,4.385104e+07,76.88,1.900,13913.839
3,Andorra,Europe,7.726500e+04,83.73,NaN,NaN
4,Angola,Africa,3.286627e+07,61.15,NaN,5819.495
...,...,...,...,...,...,...
207,Yemen,Asia,2.982597e+07,66.12,0.700,1479.147
208,Zambia,Africa,1.838396e+07,63.89,2.000,3689.251
209,Zimbabwe,Africa,1.486293e+07,61.49,1.700,1899.775
210,World,NaN,7.794799e+09,72.58,2.705,15469.207


In [ ]:
locations_df[locations_df.location == "Italy"]          # add column that has italy

merged_df = covid_df.merge(locations_df, on="location") # check location and add info about it in csv

result_df = merged_df[['date',
                       'new_cases', 
                       'total_cases', 
                       'new_deaths', 
                       'total_deaths', 
                       'new_tests', 
                       'total_tests', 
                       'cases_per_million', 
                       'deaths_per_million', 
                       'tests_per_million']]            # select columns you need

result_df.to_csv('results.csv', index=None)             # doesnt include indexes
